In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import re
from concurrent.futures import ThreadPoolExecutor
import urllib.parse
import pandas as pd
import time

headers = {'User-Agent': 'My User Agent 1.0',
           'From': 'youremail@domain.com'}

In [2]:
def url_by_author(author):
    name_uni = urllib.parse.quote(author, encoding='windows-1251')
    return f'https://www.chitai-gorod.ru/search/result.php?q={name_uni}&type=author'

In [3]:
def get_urls(author, driver): #функция для получения id-шников всех искомых книг одного автора
    cur_url = url_by_author(author)
    driver.get(cur_url)
    
    books_total = 0
    items = driver.find_elements_by_class_name("count-result__value")
    for item in items:
        books_total = int(item.text)
    
    while len(driver.find_elements_by_class_name("product-card")) != books_total:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        elem = driver.find_elements_by_css_selector(".js__show-more-cards")
        if elem:
            try:
                elem[0].click()
            except:
                time.sleep(1)

    return [int(item.get_attribute("data-product"))
           for item in driver.find_elements_by_class_name("product-card")]

In [4]:
authors = ['Фрай М.', 'Емец Д.', 'Хантер Э.']
id_list = []

with webdriver.Firefox(executable_path='C:\geckodriver.exe') as driver:
    for author in authors:
        id_list.append(get_urls(author, driver))

In [5]:
id_list = sum(id_list, [])

In [6]:
def get_html(book_id): #получение html-кода
    url_base = f'https://www.chitai-gorod.ru/catalog/book/{book_id}/'
    return requests.get(url_base, headers=headers).text

def book_id_info(book_id): #обработка html-ки
    film_html = get_html(book_id)
    soup = BeautifulSoup(film_html, 'html.parser')
    temp_rating = soup.select_one(".rating").text\
                    if soup.select_one(".rating") else "4 (9)"
    titles = []
    for item in soup.findAll('div', class_="product-prop__title"):
        titles.append(item.text.strip())
    values = []
    for item in soup.findAll('div', class_="product-prop__value"):
        values.append(item.text.strip())
    book_info = {}
    book_info['ID карточки'] = book_id
    book_info.update({
        'Название': soup.find('h1', itemprop='name').text.strip() 
            if soup.find('h1', itemprop='name') else None,
        'Автор': soup.find('a', class_='link product__author').text.strip() 
            if soup.find('a', class_='link product__author') else None,
        'Рейтинг': float(re.findall(r'\d+', temp_rating)[0]),
        'Голоса': int(re.findall(r'\d+', temp_rating)[1]),
        'Цена': float(re.findall(r'\d+', soup.find('div', class_='price').text)[0])
            if soup.find('div', class_='price') else -1
    })
    book_info.update(dict(zip(titles, values)))
    book_info.update({'Обложка': soup.select_one(".product__image").find('img')['src']
                                 if soup.select_one(".product__image") else None})
    return book_info

In [14]:
book_info = book_id_info(790009) # Обработка одной книжки
book_info 

{'ID карточки': 790009,
 'Название': 'НяпиZдинг, сэнсэе',
 'Автор': 'Фрай М.',
 'Рейтинг': 4.0,
 'Голоса': 3,
 'Цена': 315.0,
 'Серия': 'Миры Макса Фрая',
 'Издательство': 'АСТ',
 'Год издания': '2015',
 'Кол-во страниц': '288',
 'ISBN': '9785170885596',
 'Тираж': '15000',
 'Формат': '20.6 x 13 x 1.8',
 'Тип обложки': 'Твердая бумажная',
 'Возраст': '18+',
 'ID товара:': '2448997',
 'Обложка': 'https://img-gorod.ru/upload/iblock/f73/f731523a36c6c8f117560b6fa3281431.jpg'}

In [10]:
def threader(func, data, max_workers=16): 
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
            return executor.map(func, data)

In [15]:
%%time
result = [i for i in threader(book_id_info, id_list)]
df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

Wall time: 1min 25s


In [16]:
with open('hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [20]:
df.head()

,ID карточки,ID товара:,ISBN,Автор,Возраст,Год издания,Голоса,Издательство,Кол-во страниц,Название,Обложка,Переводчик,Редактор,Рейтинг,Серия,Тип обложки,Тираж,Формат,Художник,Цена
152,244296,1400705,9785367009781,Фрай М.,NaN,2009,0,"Амфора, ООО",478,"Книга для таких, как я",https://img-gorod.ru/upload/iblock/73b/73b0845...,NaN,NaN,0.0,ФРАМ,Твердая бумажная,4000,20.5 x 13.2 x 3,NaN,-1.0
151,245728,1665067,9785942783372,Фрай М.,NaN,2010,0,"Амфора, ООО",606,Волонтеры вечности,https://img-gorod.ru/upload/iblock/bd5/bd5917e...,NaN,NaN,0.0,NaN,Твердая бумажная,3000,20.7 x 13.8 x 3.8,NaN,-1.0
343,246061,1668112,9785699110018,Емец Д.,NaN,2003,0,Эксмо-Пресс,384,Таня Гроттер и Золотая Пиявка,https://img-gorod.ru/upload/iblock/236/236a031...,NaN,NaN,0.0,Таня Гроттер,Твердая бумажная,NaN,20.6 x 13.5 x 2.3,NaN,-1.0
150,246246,1801350,5942783535,Фрай М.,NaN,2003,0,"Амфора, ООО",350,Простые волшебные вещи,https://img-gorod.ru/upload/iblock/82d/82d0b11...,NaN,NaN,0.0,Лабиринты Эхо,Твердая бумажная,NaN,20.8 x 13.7 x 2.3,NaN,-1.0
149,246247,1801351,9785367016239,Фрай М.,NaN,2009,0,"Амфора, ООО",367,Темная сторона,https://img-gorod.ru/upload/iblock/a10/a1063f0...,NaN,NaN,0.0,Лабиринты Ехо,Твердая бумажная,3000,20.6 x 13.5 x 2.3,NaN,-1.0
